In [15]:
using ExpFamilyPCA
using BenchmarkTools
using Distances
using Random

Random.seed!(1);

In [16]:
n = 5
indim = 3
outdim = 2
X = rand(n, indim)

ϵ = eps()
G(θ) = exp(θ)
g(θ) = exp(θ)
F(x) = x * log(x + ϵ) - x
f(x) = log(x + ϵ)
B = Distances.gkl_divergence
Bg(x, θ) = exp(θ) - x * θ + x * log(x) * x - x
μ = 1;

In [ ]:
# TODO: add seperate times for construction, fitting, compressing, and for decompressing

# Benchmarking

## Default

In [17]:
b_default = @benchmark ExpFamilyPCA.fit!(
	PoissonEPCA($indim, $outdim),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 328 samples with 1 evaluation.
 Range (min … max):  14.114 ms … 121.739 ms  ┊ GC (min … max): 0.00% … 87.43%
 Time  (median):     14.890 ms               ┊ GC (median):    1.67%
 Time  (mean ± σ):   15.262 ms ±   5.908 ms  ┊ GC (mean ± σ):  3.36% ±  4.83%

              ▂▂▃▅▇▄█▄█▃▃ ▂▂▂                                   
  ▃▁▁▃▁▃▃▄▅▇▆▆███████████████▃▃▆▄▆▃▁▃▁▁▁▄▄▃▁▃▃▁▃▁▁▁▁▃▁▁▁▁▁▁▃▁▃ ▄
  14.1 ms         Histogram: frequency by time         16.6 ms <

 Memory estimate: 31.47 MiB, allocs estimate: 268465.

In [44]:
b_default = @benchmark ExpFamilyPCA.fit!(epca, $X; maxiter=30) setup = epca = PoissonEPCA($indim, $outdim)

BenchmarkTools.Trial: 329 samples with 1 evaluation.
 Range (min … max):  14.138 ms … 141.583 ms  ┊ GC (min … max): 0.00% … 89.53%
 Time  (median):     14.877 ms               ┊ GC (median):    2.52%
 Time  (mean ± σ):   15.230 ms ±   6.994 ms  ┊ GC (mean ± σ):  4.34% ±  5.03%

               ▁          ▁ █▇▁                                 
  ▂▁▁▂▁▃▃▃▃▇▇▇▇█▇▃▅▅▄█▇▇▆███████▆█▅▄▄▃▃▃▁▁▃▁▁▃▁▁▁▁▁▁▃▁▂▁▃▁▂▁▂▂ ▃
  14.1 ms         Histogram: frequency by time         15.9 ms <

 Memory estimate: 31.47 MiB, allocs estimate: 268462.

## EPCA 1

In [18]:
b1_Fg = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		$g,
		Val((:F, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 72 samples with 1 evaluation.
 Range (min … max):  65.667 ms … 174.302 ms  ┊ GC (min … max): 1.30% … 60.78%
 Time  (median):     67.963 ms               ┊ GC (median):    1.91%
 Time  (mean ± σ):   70.228 ms ±  12.702 ms  ┊ GC (mean ± σ):  3.97% ±  6.97%

      █ ▄▁       ▁                                              
  ▇▁▇▆█▁██▇▆▆▇▄▆▆█▁▄▇▆▆▁▁▆▁▁▄▁▁▆▁▁▆▁▄▄▁▁▄▆▁▁▁▇▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  65.7 ms         Histogram: frequency by time           77 ms <

 Memory estimate: 111.37 MiB, allocs estimate: 4570607.

In [19]:
b1_Ff = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		$f,
		Val((:F, :f));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.260 s …   1.422 s  ┊ GC (min … max): 0.90% … 8.46%
 Time  (median):     1.270 s              ┊ GC (median):    0.95%
 Time  (mean ± σ):   1.306 s ± 77.678 ms  ┊ GC (mean ± σ):  2.96% ± 3.78%

  ██   █                                                  █  
  ██▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.26 s         Histogram: frequency by time        1.42 s <

 Memory estimate: 1.80 GiB, allocs estimate: 117811010.

In [20]:
b1_F = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		Val((:F));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  707.208 ms … 822.764 ms  ┊ GC (min … max): 0.00% … 13.52%
 Time  (median):     712.448 ms               ┊ GC (median):    0.88%
 Time  (mean ± σ):   727.975 ms ±  41.910 ms  ┊ GC (mean ± σ):  2.66% ±  4.91%

  ▁▁█▁ ▁                                                      ▁  
  ████▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  707 ms           Histogram: frequency by time          823 ms <

 Memory estimate: 112.58 MiB, allocs estimate: 4582152.

In [40]:
b1_F = @benchmark ExpFamilyPCA.fit!(epca, $X; maxiter=30) setup = epca = EPCA(
    $indim,
    $outdim,
    $F,
    Val((:F));
    μ=$μ,
    ϵ=$ϵ
)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  701.540 ms … 827.224 ms  ┊ GC (min … max): 0.00% … 15.29%
 Time  (median):     706.020 ms               ┊ GC (median):    1.07%
 Time  (mean ± σ):   722.912 ms ±  46.038 ms  ┊ GC (mean ± σ):  3.13% ±  5.52%

    █                                                            
  ▆▆█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  702 ms           Histogram: frequency by time          827 ms <

 Memory estimate: 111.82 MiB, allocs estimate: 4570605.

In [21]:
b1_FG = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		$G,
		Val((:F, :G));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 59 samples with 1 evaluation.
 Range (min … max):  77.154 ms … 189.083 ms  ┊ GC (min … max): 0.00% … 59.32%
 Time  (median):     82.220 ms               ┊ GC (median):    7.60%
 Time  (mean ± σ):   85.450 ms ±  14.123 ms  ┊ GC (mean ± σ):  9.40% ±  6.93%

                 ▁▆█▁ █                                         
  ▄▁▁▁▁▁▁▁▄▁▁▁▁▇▇████▇█▇▇▄▁▄▁▇▄▇▄▁▁▁▄▄▁▁▁▁▄▁▄▇▄▁▄▄▁▄▄▁▄▄▁▁▁▄▁▄ ▁
  77.2 ms         Histogram: frequency by time           92 ms <

 Memory estimate: 112.19 MiB, allocs estimate: 4580722.

## EPCA 2

In [22]:
b2_Gg = @benchmark ExpFamilyPCA.fit!(
    EPCA(
        $indim,
        $outdim,
        $G,
        $g,
        Val((:G, :g));
        μ=$μ,
        ϵ=$ϵ
    ),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 43 samples with 1 evaluation.
 Range (min … max):  113.133 ms … 224.478 ms  ┊ GC (min … max): 0.00% … 49.27%
 Time  (median):     114.110 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   116.734 ms ±  16.831 ms  ┊ GC (mean ± σ):  2.36% ±  7.49%

  █                                                              
  █▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▁
  113 ms           Histogram: frequency by time          224 ms <

 Memory estimate: 26.67 MiB, allocs estimate: 240722.

In [23]:
b2_G = @benchmark ExpFamilyPCA.fit!(
    EPCA(
        $indim,
        $outdim,
        $G,
        Val((:G));
        μ=$μ,
        ϵ=$ϵ
    ),
    $X;
    maxiter=30,
)

BenchmarkTools.Trial: 39 samples with 1 evaluation.
 Range (min … max):  124.995 ms … 235.510 ms  ┊ GC (min … max): 0.00% … 46.82%
 Time  (median):     127.032 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   130.237 ms ±  17.362 ms  ┊ GC (mean ± σ):  3.10% ±  7.42%

  ▆█                                                             
  ██▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  125 ms           Histogram: frequency by time          236 ms <

 Memory estimate: 27.34 MiB, allocs estimate: 250836.

## EPCA 3

In [24]:
b3_Bg = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$B,
		$g,
		Val((:B, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 337 samples with 1 evaluation.
 Range (min … max):  13.892 ms … 134.394 ms  ┊ GC (min … max): 0.00% … 88.88%
 Time  (median):     14.422 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.841 ms ±   6.552 ms  ┊ GC (mean ± σ):  3.59% ±  5.01%

    ▂█▃▂      ▃▂ ▂                                              
  ▄▇████▇▄▆▆▆█████▆▇▅▄▄▅▄▄▃▃▂▂▃▂▃▁▂▂▃▁▂▃▃▁▁▂▁▁▂▂▂▃▁▂▁▂▁▁▁▁▃▁▁▂ ▃
  13.9 ms         Histogram: frequency by time         16.6 ms <

 Memory estimate: 18.09 MiB, allocs estimate: 185198.

In [25]:
b3_BG = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$B,
		$G,
		Val((:B, :G));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 204 samples with 1 evaluation.
 Range (min … max):  21.761 ms … 138.081 ms  ┊ GC (min … max): 0.00% … 83.87%
 Time  (median):     23.405 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.559 ms ±   8.218 ms  ┊ GC (mean ± σ):  7.35% ±  8.09%

   ▁█▂                ▃ ▁                                       
  ▅████▅▆▄▅▄▅▅▁▄▃▃▃▁▃▄█▇█▅▆▄▃▃▃▃▃▄▄▃▃▁▃▁▁▁▁▃▁▃▃▁▁▁▃▁▁▁▃▁▁▁▁▃▁▃ ▃
  21.8 ms         Histogram: frequency by time         30.8 ms <

 Memory estimate: 18.77 MiB, allocs estimate: 195313.

## EPCA 4

In [32]:
b4_fGg = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$f,
		$G,
        $g,
		Val((:f, :G, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 385 samples with 1 evaluation.
 Range (min … max):  12.221 ms … 132.316 ms  ┊ GC (min … max): 0.00% … 90.17%
 Time  (median):     12.723 ms               ┊ GC (median):    2.28%
 Time  (mean ± σ):   13.014 ms ±   6.104 ms  ┊ GC (mean ± σ):  3.84% ±  4.69%

     ▁▁▃▃▁      ▁▆██▂▁                                          
  ▂▄▅██████▅▅▅▆███████▇▇█▂▃▃▃▃▃▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▃▁▂▂▁▂ ▃
  12.2 ms         Histogram: frequency by time         14.2 ms <

 Memory estimate: 26.67 MiB, allocs estimate: 240774.

In [35]:
b4_fG = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$f,
		$G,
		Val((:f, :G));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 219 samples with 1 evaluation.
 Range (min … max):  18.631 ms … 184.919 ms  ┊ GC (min … max):  0.00% … 87.91%
 Time  (median):     22.197 ms               ┊ GC (median):    14.01%
 Time  (mean ± σ):   22.886 ms ±  11.743 ms  ┊ GC (mean ± σ):  11.48% ±  8.98%

   ▄     █▄                                                     
  ▇█▅▅▃▃▄██▅▃▁▃▄▃▃▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂▂ ▃
  18.6 ms         Histogram: frequency by time         47.2 ms <

 Memory estimate: 27.34 MiB, allocs estimate: 250889.

## EPCA 5

In [28]:
b5_Bgg = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$Bg,
		$g,
		Val((:Bg, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 312 samples with 1 evaluation.
 Range (min … max):  14.512 ms … 158.618 ms  ┊ GC (min … max): 0.00% … 90.03%
 Time  (median):     15.297 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.032 ms ±   8.141 ms  ┊ GC (mean ± σ):  4.20% ±  5.23%

    ▄▇▁ ▄▁▅█▄▃▄▁▁▁ ▂▁                                           
  ▃▅███▆█████████████▆▃▆▆▃▁▆▃▅▇▅▃▃▆▇▆▃▄▃▃▁▅▄▃▃▃▃▁▃▃▃▁▃▄▄▃▁▃▃▁▃ ▄
  14.5 ms         Histogram: frequency by time           18 ms <

 Memory estimate: 21.86 MiB, allocs estimate: 212951.

# Table

In [38]:
get_mean_time(benchmark) = mean(benchmark).time

data = Dict(
    # Default
    :default => get_mean_time(b_default),

    # EPCA 1
    (:F, :g) => get_mean_time(b1_Fg),
    (:F, :f) => get_mean_time(b1_Ff),
    (:F) => get_mean_time(b1_F),
    (:F, :G) => get_mean_time(b1_FG),

    # EPCA 2
    (:G, :g) => get_mean_time(b2_Gg),
    (:G) => get_mean_time(b2_G),

    # EPCA 3
    (:B, :g) => get_mean_time(b3_Bg),
    (:B, :G) => get_mean_time(b3_BG),

    # EPCA 4
    (:F, :G, :g) => get_mean_time(b4_fGg),
    (:f, :G) => get_mean_time(b4_fG),

    # EPCA 5
    (:Bg, :g) => get_mean_time(b5_Bgg)
)

Dict{Any, Float64} with 12 entries:
  :default     => 1.52624e7
  (:F, :g)     => 7.02277e7
  :F           => 7.27975e8
  (:F, :f)     => 1.3056e9
  (:B, :G)     => 2.45591e7
  (:G, :g)     => 1.16734e8
  (:F, :G)     => 8.54503e7
  (:f, :G)     => 2.28864e7
  (:B, :g)     => 1.48408e7
  (:F, :G, :g) => 1.30138e7
  (:Bg, :g)    => 1.6032e7
  :G           => 1.30237e8